In [3]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("..") / "data"

In [4]:
year = 2022
month = 1
rides = pd.read_parquet(f"{DATA_DIR}/transformed/validated_rides_{year}_{month:02d}.parquet")

In [6]:
rides.head(5)

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68


In [8]:
# Add a column with the rounded hour
rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
rides

,pickup_datetime,pickup_location_id,pickup_hour
0,2022-01-01 00:35:40,142,2022-01-01 00:00:00
1,2022-01-01 00:33:43,236,2022-01-01 00:00:00
2,2022-01-01 00:53:21,166,2022-01-01 00:00:00
3,2022-01-01 00:25:21,114,2022-01-01 00:00:00
4,2022-01-01 00:36:48,68,2022-01-01 00:00:00
...,...,...,...
2463926,2022-01-31 23:36:53,90,2022-01-31 23:00:00
2463927,2022-01-31 23:44:22,107,2022-01-31 23:00:00
2463928,2022-01-31 23:39:00,113,2022-01-31 23:00:00
2463929,2022-01-31 23:36:42,148,2022-01-31 23:00:00


In [12]:
# Group by hours and count number of rows (per location id and pickup hour)
agg_rides = rides.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()
agg_rides.rename(columns={0: 'rides'}, inplace=True)
agg_rides

,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,4,11
1,2022-01-01 00:00:00,7,6
2,2022-01-01 00:00:00,10,1
3,2022-01-01 00:00:00,12,2
4,2022-01-01 00:00:00,13,12
...,...,...,...
66863,2022-01-31 23:00:00,261,4
66864,2022-01-31 23:00:00,262,8
66865,2022-01-31 23:00:00,263,26
66866,2022-01-31 23:00:00,264,24


In [28]:
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame:
    location_ids = agg_rides['pickup_location_id'].unique()
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H'
    )
    output = pd.DataFrame()
    for location_id in tqdm(location_ids):
        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'rides']]

        # Quick way to add dates with 0 in a Series
        agg_rides_i.set_index('pickup_hour', inplace=True)
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)

        agg_rides_i['pickup_location_id'] = location_id
        output = pd.concat([output, agg_rides_i])

    output = output.reset_index().rename(columns={'index': 'pickup_hour'})
    return output

In [29]:
agg_rides_all_slots = add_missing_slots(agg_rides)
agg_rides_all_slots

100%|██████████| 257/257 [00:00<00:00, 1056.78it/s]


,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,11,4
1,2022-01-01 01:00:00,15,4
2,2022-01-01 02:00:00,26,4
3,2022-01-01 03:00:00,8,4
4,2022-01-01 04:00:00,9,4
...,...,...,...
191203,2022-01-31 19:00:00,0,176
191204,2022-01-31 20:00:00,0,176
191205,2022-01-31 21:00:00,0,176
191206,2022-01-31 22:00:00,0,176
